In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding,Bidirectional,LSTM,GlobalMaxPool1D,Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import save_model
import csv 
import pandas as pd 
from IPython.display import display, HTML
import h5py
from random import shuffle

In [0]:
%cd "drive/My Drive/Clickbait-Detection-master/clickbait17-train-170331"

/content/drive/My Drive/Clickbait-Detection-master/clickbait17-train-170331


In [0]:
truth_data = pd.read_json("truth.jsonl", lines=True)
truth_data.head()
#testing data
Ttruth_data = pd.read_json("Ttruth.jsonl", lines=True)

In [0]:
clickbait_data = pd.read_json("instances.jsonl", lines=True)
clickbait_data.head()
#test data
Tclickbait_data = pd.read_json("Tinstances.jsonl", lines=True)

In [0]:
truth_data.head()

,truthJudgments,truthMean,id,truthClass,truthMedian,truthMode
0,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,858464162594172928,clickbait,1.000000,1.000000
1,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,858462320779026432,no-clickbait,0.000000,0.000000
2,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,858460992073863168,no-clickbait,0.333333,0.000000
3,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,858459539296980992,no-clickbait,0.333333,0.333333
4,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858455355948384256,no-clickbait,0.000000,0.000000


In [0]:
clickbait_data.head()

,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription
0,[],[UK’s response to modern slavery leaving victi...,858462320779026432,[modern-slavery-rex.jpg],[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,Sat Apr 29 23:25:41 +0000 2017,"modern slavery, Department For Work And Pensio...",“Inexcusable” failures in the UK’s system for ...
1,[],[this is good],858421020331560960,"[In this July 1, 2010 file photo, Dr. Charmain...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,Sat Apr 29 20:41:34 +0000 2017,"Americans United for Life, Dr. Charmaine Yoest...",President Donald Trump has appointed pro-life ...
2,[],"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,[President Trump will not attend this year's W...,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,Sat Apr 29 17:11:23 +0000 2017,"trump whcd, whcd, white house correspondents d...",President Trump won't be at this year's White ...
3,[],[Meet the happiest #dog in the world!],858323428260139008,"[Maru , Maru, Maru, Maru, Maru]",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",Sat Apr 29 14:13:46 +0000 2017,"Maru, husky, dogs, pandas, furball, instagram","The article is about Maru, a husky dog who has..."
4,[],[Tokyo's subway is shut down amid fears over a...,858283602626347008,[All nine lines of Tokyo's subway system were ...,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,"Tokyo,subway,shut,fears,North,Korean,attack","The temporary suspension, which lasted ten min..."


In [0]:
data = pd.merge(clickbait_data,truth_data, on = 'id')
data.head()
Tdata = pd.merge(Tclickbait_data,Ttruth_data, on = 'id')

In [0]:
data['postText']

0        [UK’s response to modern slavery leaving victi...
1                                           [this is good]
2        [The "forgotten" Trump roast: Relive his bruta...
3                   [Meet the happiest #dog in the world!]
4        [Tokyo's subway is shut down amid fears over a...
                               ...                        
19533    [Brazil soccer team and pilot's final intervie...
19534                                     [😱😱😱😱😱😱😱😱😱😱😱😱😱😱]
19535    [Frenchs Forest high school may have to make w...
19536                                     [Oh Jeff… #bruh]
19537    [Richard Sherman weighs in on Cam Newton’s str...
Name: postText, Length: 19538, dtype: object

In [0]:
data['postText'][0]

['UK’s response to modern slavery leaving victims destitute while abusers go free']

In [0]:
data['targetDescription'][0]

'“Inexcusable” failures in the UK’s system for dealing with modern slavery are\xa0leaving victims reduced to destitution while their abusers go free because they are not adequately supported to testify against them, an alarming report has warned.'

In [0]:
data.shape

(19538, 14)

In [0]:
data.to_csv('data.csv')

In [0]:
Tdata.to_csv('Tdata.csv')

In [0]:
data = pd.read_csv('train1.csv')
Tdata = pd.read_csv('test1.csv')

In [0]:
data_text = data['concatenate']

In [0]:
def tokenize(thresh = 5):
    count  = dict()
    idx = 1
    word_index = dict()
    for txt in data_text:
        words = process(txt)
        for word in words:
            if word in count.keys():
                count[word] += 1
            else:
                count[word]  = 1
    most_counts = [word for word in count.keys() if count[word]>=thresh]
    for word in most_counts:
        word_index[word] = idx
        idx+=1
    return word_index

In [0]:
import re
def process(txt):
    out = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    out = out.split()
    out = [word.lower() for word in out]
    return out

In [0]:
num_words = None
word_index = tokenize()
num_words = len(word_index)
print('length of the dictionary ',len(word_index))

length of the dictionary  10935


In [0]:
def getMax(data):
    max_tokens = 0 
    for txt in data:
        if max_tokens < len(txt.split()):
            max_tokens = len(txt.split())
    return max_tokens
max_tokens = getMax(data['concatenate'])

In [0]:
max_tokens

167

In [0]:
def create_sequences(data):
    tokens = []
    for txt in data:
        words = process(txt)
        seq = [0] * max_tokens
        i = 0 
        for word in words:
            start = max_tokens-len(words)
            if word.lower() in word_index.keys():
                seq[i+start] = word_index[word]
            i+=1
        tokens.append(seq)        
    return np.array(tokens)

In [0]:
x_train_tokens1 = create_sequences(data['concatenate'])

In [0]:
x_test_tokens = create_sequences(Tdata['concatenate'])

In [0]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

model.add(GRU(units=16, name = "gru_1",return_sequences=True))
model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
model.add(GRU(units=4, name= "gru_3"))
model.add(Dense(1, activation='sigmoid',name="dense_1"))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model.fit(x_train_tokens1, data['label'], validation_split=0.05, epochs=3, batch_size=32)

Train on 17665 samples, validate on 930 samples
Epoch 1/3
17665/17665 [==============================] - 335s 19ms/sample - loss: 0.5285 - acc: 0.7749 - val_loss: 0.4694 - val_acc: 0.7946
Epoch 2/3
17665/17665 [==============================] - 342s 19ms/sample - loss: 0.4238 - acc: 0.8182 - val_loss: 0.4650 - val_acc: 0.7839
Epoch 3/3
17665/17665 [==============================] - 339s 19ms/sample - loss: 0.3545 - acc: 0.8593 - val_loss: 0.4910 - val_acc: 0.7710


In [0]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test_tokens, Tdata['label'], batch_size=32)
print('test loss, test acc:', results)


# Evaluate on test data
2267/2267 [==============================] - 9s 4ms/sample - loss: 0.6241 - acc: 0.7128
test loss, test acc: [0.6240759046265981, 0.7128363]


In [0]:
save_model(
    model,
    "keras.h5",
    overwrite=True,
    include_optimizer=True
)

In [0]:
txt = ["[title] description"]
print(create_sequences(txt)[0])
pred = model.predict(create_sequences(txt))
print('\n prediction for \n',pred[:,0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [0]:
def create_csv(file):
    with open(file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for key in word_index.keys():
            writer.writerow([key,word_index[key]])

In [0]:
create_csv('dict.csv')

In [0]:
!pip install tensorflowjs

     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 104.6MB 100kB/s 
     |████████████████████████████████| 256kB 48.2MB/s 
     |████████████████████████████████| 921kB 55.6MB/s 
     |████████████████████████████████| 450kB 43.3MB/s 
     |████████████████████████████████| 3.9MB 58.9MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=2040bf38603a8d5cdc22fef88d3dfea71da13f082c743a1a4f9f4d1407faea28
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=9236303b34cf004b3dd5e31581b79a1c30e53cb2638e2fddb4d17fb4cfed3deb
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built PyInquirer gast
ERROR: tensorflow 1.15.2 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which i

In [0]:
!tensorflowjs_converter --input_format keras keras.h5 my_model_as_tfjs1